In [13]:
%store

Stored variables and their in-db values:
account_id                                                  -> '607916531205'
alarm_name_failure                                          -> 'FlightDelayEndpointFailureAlarm'
alarm_name_latency                                          -> 'FlightDelayEndpointLatencyAlarm'
baseline_dataset_path                                       -> 's3://sagemaker-us-east-1-607916531205/data_captur
baseline_model_logistic_path                                -> 'baseline_model_logistic.pkl'
baseline_model_path                                         -> 'baseline_model.pkl'
baseline_results_uri                                        -> 's3://sagemaker-us-east-1-607916531205/baseline_re
create_base_csv_athena_db                                   -> True
create_base_csv_athena_table                                -> True
database_name                                               -> 'db_airline_delay_cause'
dev_feature_group_name                                    

In [16]:
import os
import glob
import time
import numpy as np  # ✅ Fix: Import numpy for drift threshold calculations
import boto3  # ✅ Import boto3 for AWS clients
from botocore.exceptions import ClientError

# ✅ Initialize AWS Clients
sagemaker_client = boto3.client("sagemaker")
s3_client = boto3.client("s3")
athena_client = boto3.client("athena")
glue_client = boto3.client("glue")
cw_client = boto3.client("cloudwatch")  # ✅ Added CloudWatch client

# ✅ Retrieve stored variables safely, defaulting to None
stored_variables = [
    "dev_feature_store_table", "prod_feature_store_table", 
    "dev_feature_group_name", "prod_feature_group_name",
    "baseline_model_path", "baseline_model_logistic_path",
    "endpoint_name_single_request", "endpoint_name_batch_transform",
    "flight_delay_data_quality_monitor_schedule_name",
    "flight_delay_drift_monitor_schedule_name",
    "alarm_name_latency",
    "alarm_name_failure",
    "latency_threshold",
    "failure_threshold",
    "monitor_schedule_name",
    "model_drift_alarm_name"
]

# ✅ Ensure all variables exist, setting them to None if not found in %store
for var in stored_variables:
    try:
        %store -r {var}
    except KeyError:
        print(f"⚠️ Warning: `{var}` is not stored. Defaulting to None.")
        globals()[var] = None  # Explicitly set missing variables to None

print("✅ Stored variables loaded (if available).")


✅ Stored variables loaded (if available).


In [20]:
# ✅ Delete CSV, JSON, and PKL Files from Root Directory
def delete_root_files():
    """
    Deletes all .csv, .json, and .pkl files from the root directory.
    """
    extensions_to_remove = ["csv", "json", "pkl"]
    files_deleted = 0

    for ext in extensions_to_remove:
        files = glob.glob(f'./*.{ext}')  # Only files in the root directory
        for file in files:
            try:
                os.remove(file)
                print(f"🗑️ Deleted: {file}")
                files_deleted += 1
            except Exception as e:
                print(f"❌ Error deleting {file}: {e}")

    if files_deleted == 0:
        print("✅ No CSV, JSON, or PKL files found in root directory.")
    else:
        print(f"✅ Deleted {files_deleted} files from root directory.")

# ✅ Delete Feature Groups
def delete_feature_groups():
    for feature_group_name in [dev_feature_group_name, prod_feature_group_name]:
        if not feature_group_name:
            continue  # Skip if no feature group name

        try:
            existing_groups = sagemaker_client.list_feature_groups()['FeatureGroupSummaries']
            if feature_group_name in [fg['FeatureGroupName'] for fg in existing_groups]:
                print(f"🚀 Deleting Feature Group `{feature_group_name}`...")
                sagemaker_client.delete_feature_group(FeatureGroupName=feature_group_name)
                time.sleep(5)  # Allow deletion to complete
                print(f"✅ Feature Group `{feature_group_name}` deleted.")
        except Exception as e:
            print(f"❌ Error deleting `{feature_group_name}`: {e}")

# ✅ Delete Model Files
def delete_model_files():
    for model_path in [baseline_model_path, baseline_model_logistic_path]:
        if model_path and os.path.exists(model_path):
            try:
                os.remove(model_path)
                print(f"🗑️ Deleted model file: {model_path}")
            except Exception as e:
                print(f"❌ Error deleting `{model_path}`: {e}")

# ✅ Delete SageMaker Endpoints (Single Request & Batch Transform)
def delete_sagemaker_endpoints():
    endpoint_names = [name for name in [endpoint_name_single_request, endpoint_name_batch_transform] if name is not None]

    if not endpoint_names:
        print("✅ No SageMaker endpoints to delete.")
        return

    for endpoint_name in endpoint_names:
        try:
            response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
            if response["EndpointStatus"] in ["Creating", "InService", "RollingBack", "Updating"]:
                print(f"🚀 Deleting `{endpoint_name}`...")
                sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
                time.sleep(5)  # Allow deletion to complete
                print(f"✅ Endpoint `{endpoint_name}` deleted.")
        except ClientError:
            print(f"⚠️ Endpoint `{endpoint_name}` not found. Skipping...")

# ✅ Delete SageMaker Models
def delete_sagemaker_models():
    try:
        models = sagemaker_client.list_models()["Models"]
        model_names = [model["ModelName"] for model in models]

        if model_names:
            print(f"🚀 Deleting {len(model_names)} SageMaker models...")
            for model_name in model_names:
                sagemaker_client.delete_model(ModelName=model_name)
                print(f"✅ Deleted model: {model_name}")
        else:
            print("✅ No SageMaker models found.")
    except ClientError as e:
        print(f"❌ Error deleting models: {e}")

# ✅ Stop Ongoing SageMaker Batch Transform Jobs (Completed jobs remain in history)
def stop_sagemaker_batch_jobs():
    try:
        batch_jobs = sagemaker_client.list_transform_jobs()["TransformJobSummaries"]
        running_jobs = [job["TransformJobName"] for job in batch_jobs if job["TransformJobStatus"] in ["InProgress", "Stopping"]]
        completed_jobs = [job["TransformJobName"] for job in batch_jobs if job["TransformJobStatus"] == "Completed"]

        if running_jobs:
            print(f"🚀 Stopping {len(running_jobs)} running SageMaker batch jobs...")
            for job_name in running_jobs:
                sagemaker_client.stop_transform_job(TransformJobName=job_name)
                print(f"✅ Stopped batch job: {job_name}")
        else:
            print("✅ No running batch jobs to stop.")

        if completed_jobs:
            print(f"ℹ️ {len(completed_jobs)} completed batch jobs remain in history (cannot be deleted).")

    except ClientError as e:
        print(f"❌ Error stopping batch jobs: {e}")



# ✅ Delete All Remaining S3 Files
def delete_s3_files():
    try:
        print(f"🔍 Checking S3 for files to delete in `{bucket}`...")
        objects = s3_client.list_objects_v2(Bucket=bucket)

        if "Contents" in objects:
            print(f"🚀 Deleting all {len(objects['Contents'])} files in `{bucket}`...")
            objects_to_delete = [{"Key": obj["Key"]} for obj in objects["Contents"]]
            s3_client.delete_objects(Bucket=bucket, Delete={"Objects": objects_to_delete})
            print(f"✅ All S3 files deleted.")
        else:
            print(f"✅ No files found in S3 bucket.")
    except ClientError as e:
        print(f"❌ Error deleting S3 files: {e}")



# ✅ Delete Batch Transform Files in S3
def delete_s3_batch_files():
    batch_s3_path = f"{prefix}/batch-output/"
    try:
        objects = s3_client.list_objects_v2(Bucket=bucket, Prefix=batch_s3_path)
        if "Contents" in objects:
            print(f"🚀 Deleting batch files in `{batch_s3_path}`...")
            s3_client.delete_objects(Bucket=bucket, Delete={"Objects": [{"Key": obj["Key"]} for obj in objects["Contents"]]})
            print("✅ Batch files deleted.")
    except Exception as e:
        print(f"❌ Error deleting batch files: {e}")

# ✅ Drop Feature Store Tables in Athena
def drop_feature_store_tables():
    try:
        tables = athena_client.list_table_metadata(CatalogName="AwsDataCatalog", DatabaseName=ATHENA_DATABASE)["TableMetadataList"]
        table_names = [table["Name"] for table in tables if "airline_delay_features" in table["Name"]]

        if table_names:
            print(f"🚀 Dropping {len(table_names)} tables from Athena...")
            for table in table_names:
                athena_client.start_query_execution(
                    QueryString=f"DROP TABLE IF EXISTS {ATHENA_DATABASE}.{table};",
                    QueryExecutionContext={"Database": ATHENA_DATABASE},
                    ResultConfiguration={"OutputLocation": f"s3://{bucket}/athena-query-results/"}
                )
                print(f"✅ Dropped `{table}`.")
                time.sleep(2)
        else:
            print("✅ No old feature store tables found in Athena.")
    except ClientError as e:
        print(f"❌ Error dropping Athena tables: {e}")

# ✅ Delete All Glue Databases (except `default`)
def delete_glue_databases():
    try:
        glue_databases = glue_client.get_databases()["DatabaseList"]
        glue_db_names = [db["Name"] for db in glue_databases if db["Name"] != "default"]  # Avoid deleting `default`

        if glue_db_names:
            print(f"🚀 Deleting {len(glue_db_names)} Glue databases...")
            for db in glue_db_names:
                glue_client.delete_database(Name=db)
                print(f"✅ Deleted Glue database: {db}")
        else:
            print("✅ No unnecessary Glue databases found.")
    except ClientError as e:
        print(f"❌ Error deleting Glue databases: {e}")


# ✅ Delete Feature Store S3 Files
def delete_s3_feature_store_files():
    feature_store_prefix = "feature-store/"
    try:
        objects = s3_client.list_objects_v2(Bucket=bucket, Prefix=feature_store_prefix)
        if "Contents" in objects:
            print(f"🚀 Deleting feature store files in `{feature_store_prefix}`...")
            s3_client.delete_objects(Bucket=bucket, Delete={"Objects": [{"Key": obj["Key"]} for obj in objects["Contents"]]})
            print("✅ Feature store files deleted.")
        else:
            print("✅ No feature store files found.")
    except ClientError as e:
        print(f"❌ Error deleting feature store files: {e}")

# ✅ Delete SageMaker Training & Debugger Output Files
def delete_s3_training_output():
    training_output_prefix = "flight-delay-prediction-xgboost/output/"
    
    try:
        print(f"🔍 Checking S3 for training output files in `{training_output_prefix}`...")
        response = s3_client.list_objects_v2(Bucket=bucket, Prefix=training_output_prefix)

        if "Contents" in response:
            print(f"🚀 Deleting all {len(response['Contents'])} training output files in `{training_output_prefix}`...")
            objects_to_delete = [{"Key": obj["Key"]} for obj in response["Contents"]]
            s3_client.delete_objects(Bucket=bucket, Delete={"Objects": objects_to_delete})
            print(f"✅ Training output files deleted.")
        else:
            print(f"✅ No training output files found in S3 bucket.")
    except ClientError as e:
        print(f"❌ Error deleting training output files: {e}")

# ✅ Delete Monitoring Schedules
def delete_monitoring_schedules():
    for schedule_name in [flight_delay_drift_monitor_schedule_name, flight_delay_data_quality_monitor_schedule_name]:
        if not schedule_name:
            continue  # Skip if no schedule name

        try:
            existing_schedules = sagemaker_client.list_monitoring_schedules()['MonitoringScheduleSummaries']
            if schedule_name in [schedule['MonitoringScheduleName'] for schedule in existing_schedules]:
                print(f"🚀 Deleting Monitoring Schedule `{schedule_name}`...")
                sagemaker_client.delete_monitoring_schedule(MonitoringScheduleName=schedule_name)  # ✅ Fix: Use sagemaker_client
                time.sleep(5)  # Allow deletion to complete
                print(f"✅ Monitoring Schedule `{schedule_name}` deleted.")
        except Exception as e:
            print(f"❌ Error deleting `{schedule_name}`: {e}")

# ✅ Delete CloudWatch Alarms
def delete_cloudwatch_alarms():
    alarms_to_delete = [
        alarm_name_latency,  # ✅ Fix: Correct variable name from %store
        alarm_name_failure,  # ✅ Fix: Correct variable name from %store
        model_drift_alarm_name  # ✅ Fix: Added FlightDelayModelDriftAlarm for deletion
    ]
    
    for alarm_name in alarms_to_delete:
        if alarm_name:
            try:
                print(f"🚀 Deleting CloudWatch Alarm `{alarm_name}`...")
                cw_client.delete_alarms(AlarmNames=[alarm_name])
                print(f"✅ CloudWatch Alarm `{alarm_name}` deleted.")
            except Exception as e:
                print(f"❌ Error deleting alarm `{alarm_name}`: {e}")

# ✅ Drop All Athena Tables from All Databases
def drop_feature_store_tables():
    try:
        # ✅ List all databases in Athena
        databases = athena_client.list_databases(CatalogName="AwsDataCatalog")["DatabaseList"]
        database_names = [db["Name"] for db in databases]

        for database in database_names:
            # ✅ List tables in the database
            tables = athena_client.list_table_metadata(CatalogName="AwsDataCatalog", DatabaseName=database)["TableMetadataList"]
            table_names = [table["Name"] for table in tables]

            if table_names:
                print(f"🚀 Dropping {len(table_names)} tables from Athena database `{database}`...")
                for table in table_names:
                    query = f"DROP TABLE IF EXISTS {database}.{table};"
                    athena_client.start_query_execution(
                        QueryString=query,
                        QueryExecutionContext={"Database": database},
                        ResultConfiguration={"OutputLocation": f"s3://{bucket}/athena-query-results/"}
                    )
                    print(f"✅ Dropped `{table}` from `{database}`.")
                    time.sleep(2)
            else:
                print(f"✅ No tables found in Athena database `{database}`.")

    except ClientError as e:
        print(f"❌ Error dropping Athena tables: {e}")





In [21]:
# ✅ Run Full Cleanup
def clean_state():
    print("\n🚀 **Starting Full Cleanup...**\n")

    # ✅ Step 1: Delete SageMaker-related resources in dependency order
    delete_monitoring_schedules()  # Must delete first before endpoints/models
    delete_cloudwatch_alarms()
    delete_feature_groups()
    delete_sagemaker_models()  # Must be deleted before endpoints
    delete_sagemaker_endpoints()
    stop_sagemaker_batch_jobs()  # Stop any running batch jobs

    # ✅ Step 2: Clean up storage in order of dependencies
    delete_model_files()  # Remove locally stored model files
    delete_s3_batch_files()  # Delete batch transform outputs before main S3 cleanup
    delete_s3_feature_store_files()  # Delete feature store S3 data before dropping tables

    # ✅ Step 3: Delete Athena & Glue-related resources
    drop_feature_store_tables()  # Drop tables first before deleting the database
    delete_glue_databases()  # Remove Glue databases

    # ✅ Step 4: Delete remaining S3 files (now safe to remove)
    delete_s3_training_output()
    delete_s3_files()
    delete_root_files()

    print("\n✅ **Cleanup completed successfully!**")

# ✅ Execute cleanup
clean_state()



🚀 **Starting Full Cleanup...**

🚀 Deleting CloudWatch Alarm `FlightDelayEndpointLatencyAlarm`...
✅ CloudWatch Alarm `FlightDelayEndpointLatencyAlarm` deleted.
🚀 Deleting CloudWatch Alarm `FlightDelayEndpointFailureAlarm`...
✅ CloudWatch Alarm `FlightDelayEndpointFailureAlarm` deleted.
🚀 Deleting CloudWatch Alarm `FlightDelayModelDriftAlarm`...
✅ CloudWatch Alarm `FlightDelayModelDriftAlarm` deleted.
✅ No SageMaker models found.
⚠️ Endpoint `flight-delay-xgboost-endpoint-single-request` not found. Skipping...
⚠️ Endpoint `flight-delay-xgboost-endpoint-with-batch-transform` not found. Skipping...
✅ No running batch jobs to stop.
ℹ️ 10 completed batch jobs remain in history (cannot be deleted).
✅ No feature store files found.
🚀 Dropping 3 tables from Athena database `db_airline_delay_cause`...
✅ Dropped `airline_delay_cause_csv_raw` from `db_airline_delay_cause`.
✅ Dropped `development_data` from `db_airline_delay_cause`.
✅ Dropped `production_data` from `db_airline_delay_cause`.
✅ No ta

In [26]:
# ✅ List of all stored variables to delete
all_stored_variables = [
    "dev_feature_store_table", "prod_feature_store_table", 
    "dev_feature_group_name", "prod_feature_group_name",
    "baseline_model_path", "baseline_model_logistic_path",
    "endpoint_name_single_request", "endpoint_name_batch_transform",
    "flight_delay_data_quality_monitor_schedule_name",
    "flight_delay_drift_monitor_schedule_name",
    "alarm_name_latency", "alarm_name_failure",
    "latency_threshold", "failure_threshold",
    "monitor_schedule_name", "model_drift_alarm_name",
    "account_id", "baseline_dataset_path", "baseline_results_uri",
    "create_base_csv_athena_db", "create_base_csv_athena_table",
    "database_name", "dev_s3_path", "dev_s3_uri", "dev_table_name",
    "drift_thresholds", "model_monitor", "packages_installed",
    "prod_s3_path", "prod_s3_uri", "prod_table_name",
    "raw_table_name", "region", "role",
    "s3_csv_private_path", "s3_staging_dir", "setup_s3_bucket_passed"
]

# ✅ Remove all stored variables
def clear_all_stored_variables():
    for var in all_stored_variables:
        try:
            %store -d {var}  # Delete from %store
            print(f"🧹 Removed `{var}` from %store.")
        except Exception:
            print(f"⚠️ `{var}` was not in %store. Skipping...")

# ✅ Execute cleanup
clear_all_stored_variables()

# ✅ Verify if all variables are removed
%store


🧹 Removed `dev_feature_store_table` from %store.
🧹 Removed `prod_feature_store_table` from %store.
🧹 Removed `dev_feature_group_name` from %store.
🧹 Removed `prod_feature_group_name` from %store.
🧹 Removed `baseline_model_path` from %store.
🧹 Removed `baseline_model_logistic_path` from %store.
🧹 Removed `endpoint_name_single_request` from %store.
🧹 Removed `endpoint_name_batch_transform` from %store.
🧹 Removed `flight_delay_data_quality_monitor_schedule_name` from %store.
🧹 Removed `flight_delay_drift_monitor_schedule_name` from %store.
🧹 Removed `alarm_name_latency` from %store.
🧹 Removed `alarm_name_failure` from %store.
🧹 Removed `latency_threshold` from %store.
🧹 Removed `failure_threshold` from %store.
🧹 Removed `monitor_schedule_name` from %store.
🧹 Removed `model_drift_alarm_name` from %store.
🧹 Removed `account_id` from %store.
🧹 Removed `baseline_dataset_path` from %store.
🧹 Removed `baseline_results_uri` from %store.
🧹 Removed `create_base_csv_athena_db` from %store.
🧹 Remove

# Use code below to check what else is on your system and whether something was left behind

In [22]:
import boto3
from botocore.exceptions import ClientError
from pyathena import connect

# ✅ Initialize AWS Session
session = boto3.session.Session()
region = session.region_name
sagemaker_session = boto3.Session()  # ✅ Fix: Corrected SageMaker session initialization

# ✅ Use SageMaker's default bucket
bucket = sagemaker_session.client("s3").list_buckets()["Buckets"][0]["Name"]

# ✅ Set up Athena connection
s3_staging_dir = f's3://{bucket}/athena-query-results/'
conn = connect(s3_staging_dir=s3_staging_dir, region_name=region)

# ✅ Initialize AWS clients
s3_client = boto3.client("s3", region_name=region)
athena_client = boto3.client("athena", region_name=region)
sagemaker_client = boto3.client("sagemaker", region_name=region)
glue_client = boto3.client("glue", region_name=region)
cw_client = boto3.client("cloudwatch", region_name=region)  # ✅ Added CloudWatch client

# ✅ Function to list AWS resources
def list_aws_resources():
    print("\n--- 📌 AWS Resource Overview ---")

    # ✅ List Athena Databases
    try:
        databases = athena_client.list_databases(CatalogName="AwsDataCatalog")["DatabaseList"]
        database_names = [db["Name"] for db in databases]
        print("\n📌 **Athena Databases:**")
        for db in database_names:
            print(f"   - {db}")
    except ClientError as e:
        print("❌ Error listing Athena databases:", e)
        database_names = []  # Ensure it doesn't break the next step

    # ✅ List Athena Tables Per Database
    for database in database_names:
        try:
            tables = athena_client.list_table_metadata(CatalogName="AwsDataCatalog", DatabaseName=database)["TableMetadataList"]
            table_names = [table["Name"] for table in tables]

            print(f"\n📌 **Tables in Athena Database: `{database}`**")
            if table_names:
                for table in table_names:
                    print(f"   - {table}")
            else:
                print("   ❌ No tables found in this database.")

        except ClientError as e:
            print(f"❌ Error listing tables in `{database}`:", e)

    # ✅ List Feature Store Groups
    try:
        feature_groups = sagemaker_client.list_feature_groups()["FeatureGroupSummaries"]
        feature_group_names = [fg["FeatureGroupName"] for fg in feature_groups]
        print("\n📌 **Feature Store Groups:**")
        if feature_group_names:
            for fg in feature_group_names:
                print(f"   - {fg}")
        else:
            print("   ❌ No Feature Groups found.")
    except ClientError as e:
        print("❌ Error listing Feature Groups:", e)

    # ✅ List Deployed SageMaker Endpoints
    try:
        endpoints = sagemaker_client.list_endpoints()["Endpoints"]
        endpoint_names = [ep["EndpointName"] for ep in endpoints]
        print("\n📌 **SageMaker Endpoints:**")
        if endpoint_names:
            for ep in endpoint_names:
                print(f"   - {ep}")
        else:
            print("   ❌ No SageMaker Endpoints found.")
    except ClientError as e:
        print("❌ Error listing SageMaker Endpoints:", e)

    # ✅ List Deployed SageMaker Models
    try:
        models = sagemaker_client.list_models()["Models"]
        model_names = [model["ModelName"] for model in models]
        print("\n📌 **SageMaker Models:**")
        if model_names:
            for model in model_names:
                print(f"   - {model}")
        else:
            print("   ❌ No SageMaker Models found.")
    except ClientError as e:
        print("❌ Error listing SageMaker Models:", e)

    # ✅ List SageMaker Model Monitoring Schedules
    try:
        schedules = sagemaker_client.list_monitoring_schedules()["MonitoringScheduleSummaries"]
        schedule_names = [schedule["MonitoringScheduleName"] for schedule in schedules]
        print("\n📌 **SageMaker Model Monitoring Schedules:**")
        if schedule_names:
            for schedule in schedule_names:
                print(f"   - {schedule}")
        else:
            print("   ❌ No Monitoring Schedules found.")
    except ClientError as e:
        print("❌ Error listing SageMaker Monitoring Schedules:", e)

    # ✅ List Batch Transform Jobs
    try:
        transform_jobs = sagemaker_client.list_transform_jobs()["TransformJobSummaries"]
        batch_jobs = [job["TransformJobName"] for job in transform_jobs]
        print("\n📌 **SageMaker Batch Transform Jobs:**")
        if batch_jobs:
            for job in batch_jobs:
                print(f"   - {job}")
        else:
            print("   ❌ No Batch Transform Jobs found.")
    except ClientError as e:
        print("❌ Error listing Batch Transform Jobs:", e)

    # ✅ List S3 Files
    try:
        objects = s3_client.list_objects_v2(Bucket=bucket)
        s3_files = [obj["Key"] for obj in objects.get("Contents", [])]
        print(f"\n📌 **S3 Files in Bucket `{bucket}`:**")
        if s3_files:
            for file in s3_files[:10]:  # Show only first 10 files for readability
                print(f"   - {file}")
            if len(s3_files) > 10:
                print(f"   ... ({len(s3_files)} total files)")
        else:
            print("   ❌ No files found.")
    except ClientError as e:
        print("❌ Error listing S3 files:", e)

    # ✅ List Glue Databases
    try:
        glue_databases = glue_client.get_databases()["DatabaseList"]
        glue_db_names = [db["Name"] for db in glue_databases]
        print("\n📌 **Glue Databases:**")
        if glue_db_names:
            for db in glue_db_names:
                print(f"   - {db}")
        else:
            print("   ❌ No Glue Databases found.")
    except ClientError as e:
        print("❌ Error listing Glue databases:", e)

    # ✅ List CloudWatch Alarms
    try:
        alarms = cw_client.describe_alarms()["MetricAlarms"]
        alarm_names = [alarm["AlarmName"] for alarm in alarms]
        print("\n📌 **CloudWatch Alarms:**")
        if alarm_names:
            for alarm in alarm_names:
                print(f"   - {alarm}")
        else:
            print("   ❌ No CloudWatch Alarms found.")
    except ClientError as e:
        print("❌ Error listing CloudWatch Alarms:", e)

# ✅ Run AWS resource listing
list_aws_resources()



--- 📌 AWS Resource Overview ---

📌 **Athena Databases:**
   - default

📌 **Tables in Athena Database: `default`**
   ❌ No tables found in this database.

📌 **Feature Store Groups:**
   ❌ No Feature Groups found.

📌 **SageMaker Endpoints:**
   ❌ No SageMaker Endpoints found.

📌 **SageMaker Models:**
   ❌ No SageMaker Models found.

📌 **SageMaker Model Monitoring Schedules:**
   ❌ No Monitoring Schedules found.

📌 **SageMaker Batch Transform Jobs:**
   - sagemaker-xgboost-2025-02-23-01-55-30-697
   - sagemaker-xgboost-2025-02-19-05-26-04-620
   - pipelines-lum39tvg22zt-AbaloneTransform-SdJZQXw6zR
   - pipelines-b9q304vfzej0-AbaloneTransform-hpPnA9nR73
   - pipelines-xw5ukyv44cda-AbaloneTransform-rvefu7xYmu
   - sagemaker-xgboost-2025-01-31-08-39-00-505
   - sagemaker-xgboost-2025-01-31-08-31-25-431
   - sagemaker-xgboost-2025-01-31-08-24-20-187
   - sagemaker-xgboost-2025-01-31-08-09-06-879
   - sagemaker-xgboost-2025-01-31-07-59-41-193

📌 **S3 Files in Bucket `aws-athena-query-results-